In [1]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import numpy as np
import cupy as cp
import copy
import pandas as pd
import glob

local


In [2]:
def find_legalized(N_trees, save_result=True):
    import kaggle_support as kgs
    import pack_io
    import numpy as np
    kgs.set_float32(False)
    kgs.TREE_EXPANSION = kgs.TREE_EXPANSION**2
    kgs.initialize_tree_globals()
    xyt_list = kgs.dill_load(kgs.code_dir + '../res/sols.pickle')[N_trees-1]
    sol_list = []
    scores = []
    for xyt in xyt_list:
        sol = kgs.SolutionCollectionSquare()
        sol.xyt = xyt.astype(kgs.dtype_cp)        
        try:
            _,score = pack_io.solution_list_to_dataframe([sol], return_scores= True, print_score=False)
            scores.append(score[0])
        except:
            #print('failed to score solution')
            scores.append(np.nan)
            continue
        sol_list.append(sol)
    min_score_before = np.nanmin(scores)
    sol_list = [sol for sol, score in zip(sol_list, scores) if (np.isnan(score) or score <= min_score_before + 1e-3)]
    print(f"{N_trees}: Considering {len(sol_list)} solutions")
    sol_list_legalized = []
    for sol in sol_list:
        sol_legalized = pack_io.legalize(sol, do_plot = False, validate=True, target = 1e-20, n_iter = 100, tolerance_rel_change=1e-10, line_search = False, verbose=False)
        sol_list_legalized.append(sol_legalized)
    _,scores = pack_io.solution_list_to_dataframe(sol_list_legalized, return_scores= True, print_score=False)
    print(f"{N_trees}: Score before legalize: {min_score_before:.8f}")
    print(f"{N_trees}: Score after legalize: {np.nanmin(scores):.8f}")
    if save_result:
        kgs.dill_save(sol_list_legalized[np.argmin(scores)], kgs.code_dir + f'../res/sol_legalized_{N_trees}.pickle')
    return sol_list_legalized[np.argmin(scores)]

In [3]:
# Run legalize for N=1 to 200
import multiprocessing as mp

if __name__ == '__main__':
    # Create a pool of 10 workers
    with mp.Pool(processes=10) as pool:
        # Run find_legalized for N=1 to 200 in parallel
        pool.map(find_legalized, range(1, 201))
    
    print("All legalizations completed!")

CUDARuntimeError: cudaErrorInitializationError: initialization error